In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [3]:
spark

По имеющимся данным о рейтингах фильмов (MovieLens: 100 000 рейтингов) посчитать агрегированную статистику по ним.

Задание необходимо сделать, используя RDD.

Имеются следующие входные данные:

Таблица users x movies с рейтингами. Архив с датасетом нужно скачать с сайта GroupLens. Также он загружен на HDFS в /labs/laba01/ml-100k. Файл u.data содержит все оценки, а файл u.item — список всех фильмов.
Информация о структуре датасета расположена здесь.
!hdfs dfs -ls /labs/laba01/ml-100k

id фильма для расчета индивидуальных характеристик — в Личном кабинете на странице Лабы 1.

In [6]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2022-01-06 18:46 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2022-01-06 18:46 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2022-01-06 18:46 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2022-01-06 18:46 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2022-01-06 18:46 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2022-01-06 18:46 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2022-01-06 18:46 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2022-01-06 18:46 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2022-01-06 18:46 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2022-01-06 18:46 /labs/laba01/ml-100k/u2.base


In [23]:
data = spark.sparkContext.textFile('/labs/laba01/ml-100k/u.data')
item = spark.sparkContext.textFile('/labs/laba01/ml-100k/u.item')
readme = spark.sparkContext.textFile('/labs/laba01/ml-100k/README')

In [24]:
readme.take(1000)

['SUMMARY & USAGE LICENSE',
 '=============================================',
 '',
 'MovieLens data sets were collected by the GroupLens Research Project',
 'at the University of Minnesota.',
 ' ',
 'This data set consists of:',
 '\t* 100,000 ratings (1-5) from 943 users on 1682 movies. ',
 '\t* Each user has rated at least 20 movies. ',
 '        * Simple demographic info for the users (age, gender, occupation, zip)',
 '',
 'The data was collected through the MovieLens web site',
 '(movielens.umn.edu) during the seven-month period from September 19th, ',
 '1997 through April 22nd, 1998. This data has been cleaned up - users',
 'who had less than 20 ratings or did not have complete demographic',
 'information were removed from this data set. Detailed descriptions of',
 'the data file can be found at the end of this file.',
 '',
 'Neither the University of Minnesota nor any of the researchers',
 'involved can guarantee the correctness of the data, its suitability',
 'for any particular 

'u.data     -- The full u data set, 100000 ratings by 943 users on 1682 items.',
 '              Each user has rated at least 20 movies.  Users and items are',
 '              numbered consecutively from 1.  The data is randomly',
 '              ordered. This is a tab separated list of ',
 '\t         user id | item id | rating | timestamp. ',
 '              The time stamps are unix seconds since 1/1/1970 UTC   ',
 
 'u.item     -- Information about the items (movies); this is a tab separated',
 '              list of',
 '              movie id | movie title | release date | video release date |',
 '              IMDb URL | unknown | Action | Adventure | Animation |',
 "              Children's | Comedy | Crime | Documentary | Drama | Fantasy |",
 '              Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |',
 '              Thriller | War | Western |',
 '              The last 19 fields are the genres, a 1 indicates the movie',
 '              is of that genre, a 0 indicates it is not; movies can be in',
 '              several genres at once.',
 '              The movie ids are the ones used in the u.data data set.',
 '',

In [25]:
data.take(10)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013']

In [26]:
def split_data(line):
    split = line.split('\t')
    return split[1], split[2]

In [27]:
splitted_data = data.map(split_data).cache()

In [29]:
splitted_data.take(10)

[('242', '3'),
 ('302', '3'),
 ('377', '1'),
 ('51', '2'),
 ('346', '1'),
 ('474', '4'),
 ('265', '2'),
 ('465', '5'),
 ('451', '3'),
 ('86', '3')]

In [57]:
my_film_splitted_data = splitted_data.filter(lambda x: x[0]=='302').map(lambda x: (x[1], 1))\
                                       .sortByKey(True)\
                                       .countByKey().items()

In [60]:
[v for k, v in my_film_splitted_data]

[2, 10, 46, 119, 120]

In [61]:
all_films_splitted_data = splitted_data.map(lambda x: (x[1], 1))\
                                       .sortByKey(True)\
                                       .countByKey().items()

In [62]:
[v for k, v in all_films_splitted_data]

[6110, 11370, 27145, 34174, 21201]

In [63]:
result_json = {
                "hist_film": [v for k, v in my_film_splitted_data],
                "hist_all": [v for k, v in all_films_splitted_data]
              }

In [65]:
import json
with open('lab01.json', 'w') as f:
    json.dump(result_json, f)

In [66]:
spark.stop()